In [11]:
# import necessary libraries

import pandas as pd
import numpy as np

In [12]:
# navigate to directory holding data on local machine

%cd ..
%cd data
%pwd

c:\Users\ericm\OneDrive\Documents\MADS\Capstone
c:\Users\ericm\OneDrive\Documents\MADS\Capstone\data


'c:\\Users\\ericm\\OneDrive\\Documents\\MADS\\Capstone\\data'

### Load and inspect data

In [13]:
# load data
df_raw = pd.read_csv('combined_teds_d_data.csv')

df_raw.head()
print(df_raw.shape)

(6441469, 28)


In [14]:
# examine value counts for each column
for col in df_raw:
    print(df_raw[col].value_counts())
    print('\n')

2019    1722503
2018    1666366
2017    1661207
2020    1391393
Name: DISYR, dtype: int64


6     1111588
5     1108186
7      917442
8      641430
11     585296
9      564550
4      530841
10     523624
3      187868
2      154530
12      83319
1       32795
Name: AGE, dtype: int64


 1    4154168
 2    2285004
-9       2297
Name: GENDER, dtype: int64


 5    4257631
 4    1117374
 7     485525
-9     198630
 2     157479
 8     135038
 6      40404
 9      33186
 1      14625
 3       1577
Name: RACE, dtype: int64


 1    3328416
-9    1403162
 4     766679
 2     640507
 3     302705
Name: MARSTAT, dtype: int64


 3    2789466
 2    1229036
 4    1172551
-9     551595
 5     353163
 1     345658
Name: EDUC, dtype: int64


 3    2418603
 4    2073600
 1    1031015
-9     493457
 2     424794
Name: EMPLOY, dtype: int64


 3    2086378
 4    1674662
-9    1193155
 1    1072204
 2     415070
Name: EMPLOY_D, dtype: int64


 3    3840720
 2    1044998
 1     963694
-9     592057
Name: LIV

### Define preprocessing functions

In [15]:
def replace_invalid_responses(df):
   # replace invalid responses with null values 
   for col in df:
        df[col] = df[col].replace(-9, np.nan)

    # drop rows with null values
   df = df.dropna()

   return df


In [26]:
def replace_encodings(df):
    # define response dictionary
    response_dict = {
        'AGE': {1: '12-14', 2: '15-17', 3: '18-20', 4: '21-24', 5: '25-29', 6: '30-34', 7: '35-39', 8: '40-44', 9: '45-49', 10: '50-54', 11: '55-64', 12: '65+'},
        'GENDER': {1: 'Male', 2: 'Female'},
        'RACE': {1: 'Alaskan Native', 2: 'American Indian', 3: 'Asian or Pacific Islander', 4: 'Black or African American', 5: 'White', 6: 'Asian', 7: 'Other single race', 8: 'Two or more races', 9: 'Native Hawaiian or Other Pacific Islander'},
        'MARSTAT': {1: 'Never married', 2: 'Now Married', 3: 'Separated', 4: 'Divorced, widowed'},
        'EDUC': {1: 'Less than one school grade, no schooling, nursery school, or kindergarten to Grade 8', 2: 'Grades 9 to 11', 3: 'Grade 12 (or GED)', 4: '1-3 years of college, university, or vocational school', 5: '4 years of college, university, BA/BS, some postgraduate study, or more'},
        'EMPLOY': {1: 'Full time', 2: 'Part time', 3: 'Unemployed', 4: 'Not in labor force'},
        'EMPLOY_D': {1: 'Full time', 2: 'Part time', 3: 'Unemployed', 4: 'Not in labor force'},
        'LIVARAG': {1: 'Homeless', 2: 'Dependent living', 3: 'Independent living'},
        'LIVARAG_D': {1: 'Homeless', 2: 'Dependent living', 3: 'Independent living'},
        'ARRESTS': {0: 'None', 1: 'Once', 2: 'Two or more times'},
        'ARRESTS_D': {0: 'None', 1: 'Once', 2: 'Two or more times'},
        'SERVICES': {1: 'Detox, 24-hour, hospital inpatient', 2: 'Detox, 24-hour, free-standing residential', 3: 'Rehab/residential, hospital (non-detox)', 4: 'Rehab/residential, short term (30 days or fewer)', 5: 'Rehab/residential, long term (more than 30 days)', 6: 'Ambulatory, intensive outpatient', 7: 'Ambulatory, non-intensive outpatient', 8: 'Ambulatory, detoxification'},
        'SERVICES_D': {1: 'Detox, 24-hour, hospital inpatient', 2: 'Detox, 24-hour, free-standing residential', 3: 'Rehab/residential, hospital (non-detox)', 4: 'Rehab/residential, short term (30 days or fewer)', 5: 'Rehab/residential, long term (more than 30 days)', 6: 'Ambulatory, intensive outpatient', 7: 'Ambulatory, non-intensive outpatient', 8: 'Ambulatory, detoxification'},
        'REASON': {1: 'Treatment completed', 2: 'Dropped out of treatment', 3: 'Terminated by facility', 4: 'Transferred to another treatment program or facility', 5: 'Incarcerated', 6: 'Death', 7: 'Other'},
        'LOS': {i: str(i) if i <= 30 else {31: '31-45 days', 32: '46-60 days', 33: '61-90 days', 34: '91-120 days', 35: '121-180 days', 36: '181-365 days', 37: 'greater than 365 days'}[i] for i in range(1, 38)},
        'PSOURCE': {1: 'Individual (includes self-referral)', 2: 'Alcohol/drug use care provider', 3: 'Other health care provider', 4: 'School (educational)', 5: 'Employer/EAP', 6: 'Other community referral', 7: 'Court/criminal justice referral/DUI/DWI'},
        'NOPRIOR': {0: 'No prior treatment episode', 1: 'One or more prior treatment episodes'},
        'SUB1': {1: 'None', 2: 'Alcohol', 3: 'Cocaine/crack', 4: 'Marijuana/hashish', 5: 'Heroin', 6: 'Non-prescription methadone', 7: 'Other opiates and synthetics', 8: 'PCP', 9: 'Hallucinogens', 10: 'Methamphetamine/speed', 11: 'Other amphetamines', 12: 'Other stimulants', 13: 'Benzodiazepines', 14: 'Other tranquilizers', 15: 'Barbiturates', 16: 'Other sedatives or hypnotics', 17: 'Inhalants', 18: 'Over-the-counter medications', 19: 'Other drugs'},
        'FREQ1': {1: 'No use in the past month', 2: 'Some use', 3: 'Daily use'},
        'FRSTUSE1': {1: '11 years and under', 2: '12-14 years', 3: '15-17 years', 4: '18-20 years', 5: '21-24 years', 6: '25-29 years', 7: '30 years and older'},
        'ALCDRUG': {0: 'None', 1: 'Alcohol only', 2: 'Other drugs only', 3: 'Alcohol and other drugs'},
        'DSMCRIT': {1: 'Alcohol-induced disorder', 2: 'Substance-induced disorder', 3: 'Alcohol intoxication', 4: 'Alcohol dependence', 5: 'Opioid dependence', 6: 'Cocaine dependence', 7: 'Cannabis dependence', 8: 'Other substance dependence', 9: 'Alcohol abuse', 10: 'Cannabis abuse', 11: 'Other substance abuse', 12: 'Opioid abuse', 13: 'Cocaine abuse', 14: 'Anxiety disorders', 15: 'Depressive disorders', 16: 'Schizophrenia/other psychotic disorders', 17: 'Bipolar disorders', 18: 'Attention deficit/disruptive behavior disorders', 19: 'Other mental health condition'},
        'PSYPROB': {1: 'Yes', 2: 'No'},
        'HLTHINS': {1: 'Private insurance, Blue Cross/Blue Shield, HMO', 2: 'Medicaid', 3: 'Medicare, other (e.g. TRICARE, CHAMPUS)', 4: 'None'},
        'PRIMPAY': {1: 'Self-pay', 2: 'Private insurance (Blue Cross/Blue Shield, other health insurance, workers compensation)', 3: 'Medicare', 4: 'Medicaid', 5: 'Other government payments', 6: 'No charge (free, charity, special research, teaching)', 7: 'Other'},
        'FREQ_ATND_SELF_HELP': {1: 'No attendance', 2: '1-3 times in the past month', 3: '4-7 times in the past month', 4: '8-30 times in the past month', 5: 'Some attendance, frequency is unknown'},
        'STFIPS': {
            1: 'Alabama', 2: 'Alaska', 4: 'Arizona', 5: 'Arkansas', 6: 'California', 8: 'Colorado', 9: 'Connecticut',
            10: 'Delaware', 11: 'District of Columbia', 12: 'Florida', 13: 'Georgia', 15: 'Hawaii', 16: 'Idaho', 17: 'Illinois',
            18: 'Indiana', 19: 'Iowa', 20: 'Kansas', 21: 'Kentucky', 22: 'Louisiana', 23: 'Maine', 24: 'Maryland', 25: 'Massachusetts',
            26: 'Michigan', 27: 'Minnesota', 28: 'Mississippi', 29: 'Missouri', 30: 'Montana', 31: 'Nebraska',
            32: 'Nevada', 33: 'New Hampshire', 34: 'New Jersey', 35: 'New Mexico', 36: 'New York', 37: 'North Carolina', 38: 'North Dakota',
            39: 'Ohio', 40: 'Oklahoma', 42: 'Pennsylvania', 44: 'Rhode Island', 45: 'South Carolina', 46: 'South Dakota',
            47: 'Tennessee', 48: 'Texas', 49: 'Utah', 50: 'Vermont', 51: 'Virginia', 53: 'Washington', 56: 'Wyoming', 72: 'Puerto Rico'}
        
    }

    # Map the values to the dataframe using the dictionary
    for col in df:
        if col in response_dict:
            # identify undefined values
            undefined_values = set(df[col].unique()) - set(response_dict[col].keys())
            if undefined_values:
                print(f"Undefined values in {col}: {undefined_values}")

            df[col] = df[col].map(response_dict[col]).fillna(df[col])

    return df

In [23]:
def rename_cols(df):
    # Define the column mapping
    col_mapping = {
        'DISYR': 'YEAR_OF_DISCHARGE',
        'MARSTAT': 'MARITAL_STATUS',
        'EDUC': 'EDUCATION',
        'EMPLOY': 'EMPLOYMENT_AT_ADMISSION',
        'EMPLOY_D': 'EMPLOYMENT_AT_DISCHARGE',
        'LIVARAG': 'LIVING_ARRANGEMENT_AT_ADMISSION',
        'LIVARAG_D': 'LIVING_ARRANGEMENT_AT_DISCHARGE',
        'ARRESTS': 'ARRESTS_IN_30_DAYS_PRIOR_TO_ADMISSION',
        'ARRESTS_D': 'ARRESTS_IN_30_DAYS_PRIOR_TO_DISCHARGE',
        'SERVICES': 'SERVICES_AT_ADMISSION',
        'SERVICES_D': 'SERVICES_AT_DISCHARGE',
        'REASON': 'REASON_FOR_DISCHARGE',
        'LOS': 'LENGTH_OF_STAY',
        'PSOURCE': 'PRIMARY_SOURCE_OF_REFERRAL',
        'NOPRIOR': 'PRIOR_TREATMENT_EPISODES',
        'SUB1': 'PRIMARY_SUBSTANCE_ABUSE',
        'FREQ1': 'FREQUENCY_OF_USE',
        'FRSTUSE1': 'AGE_AT_FIRST_USE',
        'ALCDRUG': 'ALCOHOL_OR_DRUG_ABUSE',
        'DSMCRIT': 'DSM_DIAGNOSIS',
        'PSYPROB': 'PSYCHIATRIC_PROBLEM',
        'HLTHINS': 'HEALTH_INSURANCE',
        'PRIMPAY': 'PRIMARY_PAYMENT_METHOD',
        'FREQ_ATND_SELF_HELP': 'FREQUENCY_OF_SELF_HELP_ATTENDANCE',
        'STFIPS': 'STATE'
    }
    
    # Rename the columns using the mapping
    df = df.rename(columns=col_mapping)

    return df


In [24]:
# Define function to perform all preprocessing steps

def preprocess_data(input_df):
    df = input_df.copy()
    df = replace_invalid_responses(df)
    df = replace_encodings(df)
    df = rename_cols(df)
    
    return df

### Preprocess raw data

In [27]:
df = preprocess_data(df_raw)

print(df.shape)

for col in df:
    print(df[col].value_counts())
    print('\n')

(1035841, 28)
2019    281437
2018    269992
2020    242636
2017    241776
Name: YEAR_OF_DISCHARGE, dtype: int64


30-34    184761
25-29    182721
35-39    156185
40-44    105056
21-24     88524
45-49     88142
55-64     83037
50-54     78231
18-20     29479
15-17     23903
65+       11098
12-14      4704
Name: AGE, dtype: int64


Male      635746
Female    400095
Name: GENDER, dtype: int64


White                                        777770
Black or African American                    148148
American Indian                               37835
Other single race                             27701
Two or more races                             16477
Native Hawaiian or Other Pacific Islander     12810
Alaskan Native                                 8696
Asian                                          6401
Asian or Pacific Islander                         3
Name: RACE, dtype: int64


Never married        655166
Divorced, widowed    180198
Now Married          135734
Separated             6474

### Write cleaned dataframe to csv for use in dashboard

In [29]:
# write to csv
df.to_csv('cleaned_data.csv', index=False)